In [41]:
#install packages
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


In [65]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
connection_string = config["connectionString"]

# Set up connection string for Azure storage and container name
connection_string_azure_storage = config["connectionString"]
container_azure = 'watertoxicity'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string_azure_storage)

# Get the container client
container_client = blob_service_client.get_container_client(container_azure)


water_toxicity_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Display the head of the DataFrame
    print(df.head())
    # If I have only one csv, I am doing the following instructions
    water_toxicity_df = df.copy()

watertoxicity.csv
                    Program                                     ParentProject  \
0  Total Maximum Daily Load  Chollas Creek Diazinon and Dissolved Metals TMDL   
1  Total Maximum Daily Load  Chollas Creek Diazinon and Dissolved Metals TMDL   
2  Total Maximum Daily Load  Chollas Creek Diazinon and Dissolved Metals TMDL   
3  Total Maximum Daily Load  Chollas Creek Diazinon and Dissolved Metals TMDL   
4  Total Maximum Daily Load  Chollas Creek Diazinon and Dissolved Metals TMDL   

                                            Project  \
0  Chollas Creek Diazinon and Dissolved Metals TMDL   
1  Chollas Creek Diazinon and Dissolved Metals TMDL   
2  Chollas Creek Diazinon and Dissolved Metals TMDL   
3  Chollas Creek Diazinon and Dissolved Metals TMDL   
4  Chollas Creek Diazinon and Dissolved Metals TMDL   

                    StationName StationCode                 SampleDate  \
0  Chollas Creek North Fork MLS      SD8(1)  2023-02-26 00:00:00+00:00   
1  Chollas Creek

/var/folders/z_/vwk5bc2d7jxd_xbknt7zmjx40000gn/T/ipykernel_10590/2047754588.py:31: DtypeWarning: Columns (25,29,44,56,59,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


In [66]:
water_toxicity_df.head()
min_date = water_toxicity_df['SampleDate'].min()
max_date = water_toxicity_df['SampleDate'].max()

print(min_date)
print(max_date)


2023-01-03 00:00:00+00:00
2023-10-18 00:00:00+00:00


In [30]:
water_toxicity_df['SampleDate'] = pd.to_datetime(water_toxicity_df['SampleDate']).dt.date
water_toxicity_df['CollectionTime'] = pd.to_datetime(water_toxicity_df['CollectionTime']).dt.time
water_toxicity_df['SampleDate'] = pd.to_datetime(water_toxicity_df['SampleDate']).dt.tz_localize(None)
water_toxicity_df['UpstreamLength'] = water_toxicity_df['UpstreamLength'].fillna(0)
water_toxicity_df['TotalReach'] = water_toxicity_df['TotalReach'].fillna(0)
water_toxicity_df['DownstreamLength'] = water_toxicity_df['DownstreamLength'].fillna(0)
water_toxicity_df['CalculatedValue'] = water_toxicity_df['CalculatedValue'].fillna(0)
water_toxicity_df['PercentEffect'] = water_toxicity_df['PercentEffect'].fillna(0)
water_toxicity_df['CollectionDepth'] = water_toxicity_df['CollectionDepth'].fillna(0)
water_toxicity_df['CollectionDepth'] = np.floor(water_toxicity_df['CollectionDepth']).astype('Int64')
water_toxicity_df.head()



,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,UnitCollectionDepth,...,UpstreamLength,DownstreamLength,TotalReach,CalculatedValue,PercentEffect,SampleID,DW_AnalyteName,DataQuality,DataQualityIndicator,Datum
0,Total Maximum Daily Load,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek North Fork MLS,SD8(1),2023-02-26,06:58:00,Not Recorded,-88,m,...,0.0,0.0,0.0,0.0,0.0,2022-W3-SD8(1)-C-01,Alkalinity as CaCO3,Unknown data quality,BatchVerificationCode:NR,WGS84
1,Total Maximum Daily Load,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek North Fork MLS,SD8(1),2023-02-26,06:58:00,Not Recorded,-88,m,...,0.0,0.0,0.0,0.0,0.0,2022-W3-SD8(1)-C-01,Ammonia as NH3,Reject record,ResultQualCode:NR,WGS84
2,Total Maximum Daily Load,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek North Fork MLS,SD8(1),2023-02-26,06:58:00,Not Recorded,-88,m,...,0.0,0.0,0.0,0.0,0.0,2022-W3-SD8(1)-C-01,ElectricalConductivity,Unknown data quality,BatchVerificationCode:NR,WGS84
3,Total Maximum Daily Load,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek North Fork MLS,SD8(1),2023-02-26,06:58:00,Not Recorded,-88,m,...,0.0,0.0,0.0,0.0,0.0,2022-W3-SD8(1)-C-01,"Oxygen, Dissolved",Unknown data quality,BatchVerificationCode:NR,WGS84
4,Total Maximum Daily Load,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek Diazinon and Dissolved Metals TMDL,Chollas Creek North Fork MLS,SD8(1),2023-02-26,06:58:00,Not Recorded,-88,m,...,0.0,0.0,0.0,0.0,0.0,2022-W3-SD8(1)-C-01,"Oxygen, Dissolved",Unknown data quality,BatchVerificationCode:NR,WGS84


In [51]:
# Load the station lookup from the provided Excel file
lookup_df = pd.read_excel('/Users/wasikulislam/Desktop/Baruch/Baruch Spring Semester 2024/CIS 4400/Cis 4400 HW /HW #2/Lookups/StationLookUp4252024214433.xlsx')
lookup_df.head()


,StationCode,StationName,StationSource,CoordinateSource,CoordinateNumber,TargetLatitude,TargetLongitude,Datum,LastUpdateDate
0,0,"0-Huntington State Beach, Orange",SCCWRP_BeachWatch,NaN,1.0,33.6293,-117.96,NaN,1900-01-01
1,0_Rapid_Microbial_2009,Rapid_Microbial_2009_0,SCCWRP_Rapid_Microbial_2009,NaN,NaN,NaN,NaN,NaN,2012-05-17
2,0000,LabQA,All,NaN,1.0,-88.0000,-88.00,NaN,2012-04-17
3,000AGRR01,Auglaize River ~7 miles SSE of Canton,DFW-ABL,GPS,1.0,-88.0000,-88.00,NAD83,2012-07-16
4,000BBC001,Big Bureau Creek 1,DFW-ABL,GPS,1.0,-88.0000,-88.00,NAD83,2014-09-04


In [74]:
# Load your lookup DataFrame, assuming lookup_df is already defined and loaded
station_lookup_df = lookup_df[['StationCode', 'StationName', 'TargetLatitude', 'TargetLongitude', 'Datum']]

station_lookup_df.insert(0, 'station_id', range(1, len(station_lookup_df) + 1))

# Define a rename mapping to the desired column names
rename_mapping = {
    'StationCode': 'stationcode',
    'StationName': 'stationname',
    'TargetLatitude': 'latitude',
    'TargetLongitude': 'longitude',
    'Datum': 'datum'
}

# Rename the columns using the mapping
station_lookup_df.rename(columns=rename_mapping, inplace=True)

station_lookup_df = station_lookup_df.dropna()
station_lookup_df.reset_index(drop=True, inplace=True)
station_lookup_df['station_id'] = range(1, len(station_lookup_df) + 1)

# Reorder the columns to place 'station_id' at the front if desired
cols = ['station_id'] + [col for col in station_lookup_df.columns if col != 'station_id']
station_lookup_df = station_lookup_df[cols]

# Now, let's display the first few rows to confirm the changes
station_lookup_df.head()

/var/folders/z_/vwk5bc2d7jxd_xbknt7zmjx40000gn/T/ipykernel_10590/1971169050.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_lookup_df.rename(columns=rename_mapping, inplace=True)


,station_id,stationcode,stationname,latitude,longitude,datum
0,1,000AGRR01,Auglaize River ~7 miles SSE of Canton,-88.0,-88.0,NAD83
1,2,000BBC001,Big Bureau Creek 1,-88.0,-88.0,NAD83
2,3,000BBC002,Big Bureau Creek 2,-88.0,-88.0,NAD83
3,4,000BBC003,Big Bureau Creek 3,-88.0,-88.0,NAD83
4,5,000BBC004,Big Bureau Creek 4,-88.0,-88.0,NAD83


In [67]:
import calendar
from datetime import datetime

# Define the start and end dates for the date range
start_date = '2023-01-03'
end_date = '2023-10-18'

# Generate the date range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
date_dimension = pd.DataFrame(date_range, columns=['date'])

# Extract year, quarter, month, day, and weekday
date_dimension['year'] = date_dimension['date'].dt.year
date_dimension['quarter'] = date_dimension['date'].dt.quarter
date_dimension['month'] = date_dimension['date'].dt.month
date_dimension['day'] = date_dimension['date'].dt.day
date_dimension['weekday'] = date_dimension['date'].dt.day_name()

# Function to calculate the week of the month
def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(np.ceil(adjusted_dom/7.0))

# Calculate week of the month and week of the year
date_dimension['week_of_month'] = date_dimension['date'].apply(week_of_month)
date_dimension['week_of_year'] = date_dimension['date'].dt.isocalendar().week

# Set 'date_id' as an incrementing integer starting from 0 or 1, based on your requirement
date_dimension['date_id'] = range(1, len(date_dimension) + 1)

# Select only the columns as per the requirement
date_dimension = date_dimension[['date_id', 'date', 'year', 'quarter', 'month', 'day', 'weekday']]

# Optionally, rename the columns to match any specific naming conventions needed
date_dimension.rename(columns={
    'date': 'date',
    'year': 'year',
    'quarter': 'quarter',
    'month': 'month',
    'day': 'day',
    'weekday': 'weekday'
}, inplace=True)

# Display the DataFrame
print(date_dimension.head())

   date_id       date  year  quarter  month  day    weekday
0        1 2023-01-03  2023        1      1    3    Tuesday
1        2 2023-01-04  2023        1      1    4  Wednesday
2        3 2023-01-05  2023        1      1    5   Thursday
3        4 2023-01-06  2023        1      1    6     Friday
4        5 2023-01-07  2023        1      1    7   Saturday
